# 第3章: ツールを使うシングルエージェント

## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。

### ライブラリのインストール

このセルは、LangGraphおよび関連するLangChainライブラリをインストールします。実行には数分かかる場合があります。
ご利用になるLLMプロバイダーに応じて、コメントアウトを解除して必要なライブラリをインストールしてください。

In [ ]:
# === ライブラリのインストール ===
# ご利用になるLLMプロバイダーに応じて、以下のコメントを解除して実行してください。
# !%pip install -qU langchain langgraph langchain_openai langchain_google_vertexai langchain_google_genai langchain_anthropic langchain_aws boto3

### LLMプロバイダーの選択

このセルでは、使用するLLMプロバイダーを選択します。
`LLM_PROVIDER` 変数に、利用したいプロバイダー名を設定してください。
選択可能なプロバイダー: `"openai"`, `"azure"`, `"google"` (Vertex AI), `"google_genai"` (Gemini API), `"anthropic"`, `"bedrock"`

In [ ]:
# === LLMプロバイダーの選択 ===
# 利用したいLLMプロバイダーを以下の変数で指定してください。
# "openai", "azure", "google" (Vertex AI), "google_genai" (Gemini API), "anthropic", "bedrock" のいずれかを選択できます。
LLM_PROVIDER = "openai"  # 例: OpenAI を利用する場合

### APIキー/環境変数の設定

以下のセルを実行する前に、選択したLLMプロバイダーに応じたAPIキーまたは環境変数を設定する必要があります。

**一般的な手順:**
1.  `.env.sample` ファイルをコピーして `.env` ファイルを作成します。
2.  `.env` ファイルを開き、選択したLLMプロバイダーに対応するAPIキーや必要な情報を記述します。
    *   **OpenAI:** `OPENAI_API_KEY`
    *   **Azure OpenAI:** `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT`, `OPENAI_API_VERSION`, `AZURE_OPENAI_DEPLOYMENT_NAME`
    *   **Google (Vertex AI):** `GOOGLE_CLOUD_PROJECT_ID`, `GOOGLE_CLOUD_LOCATION` (Colab環境外で実行する場合、`GOOGLE_APPLICATION_CREDENTIALS` 環境変数の設定も必要になることがあります)
    *   **Google (Gemini API):** `GOOGLE_API_KEY`
    *   **Anthropic:** `ANTHROPIC_API_KEY`
    *   **AWS Bedrock:** `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION_NAME` (IAMロールを使用する場合は、これらのキー設定は不要な場合がありますが、リージョン名は必須です)
3.  ファイルを保存します。

**Google Colab を使用している場合:**
上記の `.env` ファイルを使用する代わりに、Colabのシークレットマネージャーに必要なキーを登録してください。
例えば、OpenAIを使用する場合は `OPENAI_API_KEY` という名前でシークレットを登録します。
Vertex AI を利用する場合は、Colab上での認証 (`google.colab.auth.authenticate_user()`) が実行されます。

このセルは、設定された情報に基づいて環境変数をロードし、LLMクライアントを初期化します。

In [ ]:
# === APIキー/環境変数の設定 ===
import os
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む (存在する場合)
load_dotenv()

try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# --- OpenAI ---
if LLM_PROVIDER == "openai":
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY and IS_COLAB:
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 OPENAI_API_KEY を設定するか、Colab環境の場合はシークレットに OPENAI_API_KEY を設定してください。")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# --- Azure OpenAI ---
elif LLM_PROVIDER == "azure":
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
    OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if IS_COLAB:
        if not AZURE_OPENAI_API_KEY: AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
        if not AZURE_OPENAI_ENDPOINT: AZURE_OPENAI_ENDPOINT = userdata.get("AZURE_OPENAI_ENDPOINT")
        if not OPENAI_API_VERSION: OPENAI_API_VERSION = userdata.get("OPENAI_API_VERSION") # 例: "2023-07-01-preview"
        if not AZURE_OPENAI_DEPLOYMENT_NAME: AZURE_OPENAI_DEPLOYMENT_NAME = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if not AZURE_OPENAI_API_KEY: raise ValueError("Azure OpenAI APIキー (AZURE_OPENAI_API_KEY) が設定されていません。")
    if not AZURE_OPENAI_ENDPOINT: raise ValueError("Azure OpenAI エンドポイント (AZURE_OPENAI_ENDPOINT) が設定されていません。")
    if not OPENAI_API_VERSION: OPENAI_API_VERSION = "2023-07-01-preview" # デフォルトを設定することも可能
    if not AZURE_OPENAI_DEPLOYMENT_NAME: raise ValueError("Azure OpenAI デプロイメント名 (AZURE_OPENAI_DEPLOYMENT_NAME) が設定されていません。")

    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
    os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION

# --- Google Cloud Vertex AI (Gemini) ---
elif LLM_PROVIDER == "google":
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT_ID") # .env 用に修正
    LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")

    if IS_COLAB:
        if not PROJECT_ID: PROJECT_ID = userdata.get("GOOGLE_CLOUD_PROJECT_ID")
        if not LOCATION: LOCATION = userdata.get("GOOGLE_CLOUD_LOCATION") # 例: "us-central1"
        from google.colab import auth as google_auth
        google_auth.authenticate_user() # Vertex AI を使う場合は Colab での認証を推奨
    else: # Colab外の場合、.envから読み込んだ値で環境変数を設定
        if PROJECT_ID: os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID # Vertex AI SDKが参照する標準的な環境変数名
        if LOCATION: os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION

    if not PROJECT_ID: raise ValueError("Google Cloud Project ID が設定されていません。環境変数 GOOGLE_CLOUD_PROJECT_ID を設定するか、Colab環境の場合はシークレットに GOOGLE_CLOUD_PROJECT_ID を設定してください。")
    if not LOCATION: LOCATION = "us-central1" # デフォルトロケーション

# --- Google Gemini API (langchain-google-genai) ---
elif LLM_PROVIDER == "google_genai":
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY and IS_COLAB:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Google APIキーが設定されていません。環境変数 GOOGLE_API_KEY を設定するか、Colab環境の場合はシークレットに GOOGLE_API_KEY を設定してください。")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# --- Anthropic (Claude) ---
elif LLM_PROVIDER == "anthropic":
    ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY and IS_COLAB:
        ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY:
        raise ValueError("Anthropic APIキーが設定されていません。環境変数 ANTHROPIC_API_KEY を設定するか、Colab環境の場合はシークレットに ANTHROPIC_API_KEY を設定してください。")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# --- Amazon Bedrock (Claude) ---
elif LLM_PROVIDER == "bedrock":
    AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
    AWS_REGION_NAME = os.environ.get("AWS_REGION_NAME")

    if IS_COLAB: 
        if not AWS_ACCESS_KEY_ID: AWS_ACCESS_KEY_ID = userdata.get("AWS_ACCESS_KEY_ID")
        if not AWS_SECRET_ACCESS_KEY: AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
        if not AWS_REGION_NAME: AWS_REGION_NAME = userdata.get("AWS_REGION_NAME")

    if not AWS_REGION_NAME:
         raise ValueError("AWSリージョン名 (AWS_REGION_NAME) が設定されていません。Bedrock利用にはリージョン指定が必要です。")

    # 環境変数に設定 (boto3がこれらを自動で読み込む)
    if AWS_ACCESS_KEY_ID: os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
    if AWS_SECRET_ACCESS_KEY: os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
    os.environ["AWS_DEFAULT_REGION"] = AWS_REGION_NAME # boto3が参照する標準的なリージョン環境変数名
    os.environ["AWS_REGION"] = AWS_REGION_NAME # いくつかのライブラリはこちらを参照することもある

print(f"APIキー/環境変数の設定完了 (プロバイダー: {LLM_PROVIDER})")

### LLMクライアントの初期化

このセルは、上で選択・設定したLLMプロバイダーに基づいて、対応するLLMクライアントを初期化します。

In [ ]:
# === LLMクライアントの動的初期化 ===
llm = None

if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif LLM_PROVIDER == "azure":
    from langchain_openai import AzureChatOpenAI
    llm = AzureChatOpenAI(
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"), # 環境変数から取得
        openai_api_version=os.environ.get("OPENAI_API_VERSION"), # 環境変数から取得
        temperature=0,
    )
elif LLM_PROVIDER == "google":
    from langchain_google_vertexai import ChatVertexAI
    # PROJECT_ID, LOCATION は前のセルで環境変数に設定済みか、Colabの場合は直接利用
    llm = ChatVertexAI(model_name="gemini-1.5-flash-001", temperature=0, project=os.environ.get("GOOGLE_CLOUD_PROJECT"), location=os.environ.get("GOOGLE_CLOUD_LOCATION"))
elif LLM_PROVIDER == "google_genai":
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0, convert_system_message_to_human=True)
elif LLM_PROVIDER == "anthropic":
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
elif LLM_PROVIDER == "bedrock":
    from langchain_aws import ChatBedrock # langchain_community.chat_models から langchain_aws に変更の可能性あり
    # AWS_REGION_NAME は前のセルで環境変数 AWS_DEFAULT_REGION に設定済み
    llm = ChatBedrock( # BedrockChat ではなく ChatBedrock が一般的
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        # region_name=os.environ.get("AWS_DEFAULT_REGION"), # 通常、boto3が環境変数から自動で読み込む
        model_kwargs={"temperature": 0},
    )
else:
    raise ValueError(
        f"Unsupported LLM_PROVIDER: {LLM_PROVIDER}. "
        "Please choose from 'openai', 'azure', 'google', 'google_genai', 'anthropic', or 'bedrock'."
    )

print(f"LLM Provider: {LLM_PROVIDER}")
if llm:
    print(f"LLM Client Type: {type(llm)}")
    # モデル名取得の試行を汎用的に
    model_attr = getattr(llm, 'model', None) or \ 
                 getattr(llm, 'model_name', None) or \ 
                 getattr(llm, 'model_id', None) or \ 
                 (hasattr(llm, 'llm') and getattr(llm.llm, 'model', None)) # 一部のLLMクライアントのネスト構造に対応
    if hasattr(llm, 'azure_deployment') and not model_attr: # Azure特有の属性
        model_attr = llm.azure_deployment
        
    if model_attr:
        print(f"LLM Model: {model_attr}")
    else:
        print("LLM Model: (Could not determine model name from client attributes)")


---

この章では、ToolNodeの活用やPlan-and-Execute型エージェントの構築など、ツール連携を含むシングルエージェントの作成方法を学びます。

In [ ]:
# 今後の実装に向けた空のコードセル